# PySpark SparkSQL Analytics Functions

* [SQL Summer Camp: Analytic Functions | Kaggle](https://www.youtube.com/watch?v=q1aL1XH69pQ)

In [1]:
%%html
<style>
table {float:left}
</style>

In [2]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

In [3]:
import os
import sys
import gc
from datetime import (
    datetime,
    date
)

#  Environemnt Variables

## Hadoop

In [4]:
os.environ['HADOOP_CONF_DIR'] = "/opt/hadoop/hadoop-3.2.2/etc/hadoop"

In [5]:
%%bash
export HADOOP_CONF_DIR="/opt/hadoop/hadoop-3.2.2/etc/hadoop"
ls $HADOOP_CONF_DIR | head -n 5

capacity-scheduler.xml
configuration.xsl
container-executor.cfg
core-site.xml
core-site.xml.48132.2022-02-15@12:29:41~


## PYTHONPATH

Refer to the **pyspark** modules to load from the ```$SPARK_HOME/python/lib``` in the Spark installation.

* [PySpark Getting Started](https://spark.apache.org/docs/latest/api/python/getting_started/install.html)

> Ensure the SPARK_HOME environment variable points to the directory where the tar file has been extracted. Update PYTHONPATH environment variable such that it can find the PySpark and Py4J under SPARK_HOME/python/lib. One example of doing this is shown below:

```
export PYTHONPATH=$(ZIPS=("$SPARK_HOME"/python/lib/*.zip); IFS=:; echo "${ZIPS[*]}"):$PYTHONPATH
```

Alternatively install **pyspark** with pip or conda locally which installs the Spark runtime libararies (for standalone).

* [Can PySpark work without Spark?](https://stackoverflow.com/questions/51728177/can-pyspark-work-without-spark)

> As of v2.2, executing pip install pyspark will install Spark. If you're going to use Pyspark it's clearly the simplest way to get started. On my system Spark is installed inside my virtual environment (miniconda) at lib/python3.6/site-packages/pyspark/jars  
> PySpark has a Spark installation installed. If installed through pip3, you can find it with pip3 show pyspark. Ex. for me it is at ~/.local/lib/python3.8/site-packages/pyspark. This is a standalone configuration so it can't be used for managing clusters like a full Spark installation.

In [6]:
# os.environ['PYTHONPATH'] = "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip:/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
sys.path.extend([
    "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip",
    "/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
])

## PySpark packages

Execute after the PYTHONPATH setup

In [7]:
import pyspark.sql 
from pyspark.sql.types import *
from pyspark.sql.functions import (
    col,
    avg,
    stddev,
    isnan,
    to_date,
    to_timestamp,
    hour,
)

# Data

Student schema from [Oracle SQL by Example](https://learning.oreilly.com/library/view/oracle-sql-by/9780137047345/ch06.html) located in ```./data/student```. 

In [8]:
%%bash
cd ./data/student
unzip -o student.zip

Archive:  student.zip
  inflating: COURSE_DATA_TABLE.csv   
  inflating: COURSE_REVENUE_DATA_TABLE.csv  
  inflating: EMPLOYEE_DATA_TABLE.csv  
  inflating: ENROLLMENT_DATA_TABLE.csv  
  inflating: GRADE_DATA_TABLE.csv    
  inflating: GRADE_TYPE_DATA_TABLE.csv  
  inflating: INSTRUCTOR_DATA_TABLE.csv  
  inflating: SECTION_DATA_TABLE.csv  
  inflating: SECTION_HISTORY_DATA_TABLE.csv  
  inflating: STUDENT_DATA_TABLE.csv  
  inflating: ZIPCODE_DATA_TABLE.csv  


In [9]:
%%bash
cd data/student/
hdfs dfs -mkdir -p student
hdfs dfs -put -f *.csv student

rm -rf *.csv

---
# Spark Session


In [10]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder\
    .master('yarn') \
    .config('spark.submit.deployMode', 'client') \
    .config('spark.debug.maxToStringFields', 100) \
    .config('spark.executor.memory', '2g') \
    .getOrCreate()

2022-02-20 17:44:28,578 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-02-20 17:44:31,714 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [12]:
NUM_CORES = 4
NUM_PARTITIONS = 3

spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set('spark.sql.legacy.timeParserPolicy', 'LEGACY')

---

# Stduent schema CSV 

* [SparkSQL CSV Files](https://spark.apache.org/docs/latest/sql-data-sources-csv.html)

> Spark SQL provides spark.read().csv("file_name") to read a file or directory of files in CSV format into Spark DataFrame, and dataframe.write().csv("path") to write to a CSV file. Function option() can be used to customize the behavior of reading or writing.

[SparkSession.read()](https://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/SparkSession.html#read--) returns [DataFrameReader](https://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/DataFrameReader.html) instance which has [option](https://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/DataFrameReader.html#option-java.lang.String-boolean-) method by which we can specify CSV options.

The options are listed in [Data Source Option](https://spark.apache.org/docs/latest/sql-data-sources-csv.html#data-source-option)

In [85]:
course = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/COURSE_DATA_TABLE.csv")\
    .withColumn("CREATED_DATE", to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE", to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

course.printSchema()
course.createOrReplaceTempView("course")
course.show(3, truncate=False)

root
 |-- COURSE_NO: integer (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- COST: integer (nullable = true)
 |-- PREREQUISITE: integer (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+---------+----------------------------+----+------------+----------+------------+-----------+-------------+
|COURSE_NO|DESCRIPTION                 |COST|PREREQUISITE|CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+---------+----------------------------+----+------------+----------+------------+-----------+-------------+
|10       |Technology Concepts         |1195|null        |DSCHERER  |2007-03-29  |ARISCHER   |2007-04-05   |
|20       |Intro to Information Systems|1195|null        |DSCHERER  |2007-03-29  |ARISCHER   |2007-04-05   |
|25       |Intro to Programming        |1195|140         |DSCHERER  |2007-03-29  |ARISCHER   |2007-04-05   |
+--

In [162]:
course_revenue = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/COURSE_REVENUE_DATA_TABLE.csv")

course_revenue.printSchema()
course_revenue.createOrReplaceTempView("course_revenue")
course_revenue.show(3, truncate=False)

root
 |-- COURSE_NO: integer (nullable = true)
 |-- REVENUE: integer (nullable = true)
 |-- COURSE_FEE: integer (nullable = true)
 |-- NUM_ENROLLED: integer (nullable = true)
 |-- NUM_OF_SECTIONS: integer (nullable = true)

+---------+-------+----------+------------+---------------+
|COURSE_NO|REVENUE|COURSE_FEE|NUM_ENROLLED|NUM_OF_SECTIONS|
+---------+-------+----------+------------+---------------+
|10       |1195   |1195      |1           |1              |
|20       |10755  |1195      |9           |4              |
|25       |53775  |1195      |45          |8              |
+---------+-------+----------+------------+---------------+
only showing top 3 rows



In [36]:
section = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/SECTION_DATA_TABLE.csv")\
    .withColumn("START_DATE_TIME", to_date(col('START_DATE_TIME'), "dd-MMM-yy"))\
    .withColumn("CREATED_DATE", to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE", to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

section.printSchema()
section.createOrReplaceTempView("section")
section.show(3)

root
 |-- SECTION_ID: integer (nullable = true)
 |-- COURSE_NO: integer (nullable = true)
 |-- SECTION_NO: integer (nullable = true)
 |-- START_DATE_TIME: date (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- INSTRUCTOR_ID: integer (nullable = true)
 |-- CAPACITY: integer (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+----------+---------+----------+---------------+--------+-------------+--------+----------+------------+-----------+-------------+
|SECTION_ID|COURSE_NO|SECTION_NO|START_DATE_TIME|LOCATION|INSTRUCTOR_ID|CAPACITY|CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+----------+---------+----------+---------------+--------+-------------+--------+----------+------------+-----------+-------------+
|        79|      350|         3|     2007-04-14|    L509|          107|      25|  CBRENNAN|  2007-01-02|   CBRENNAN|   2007-01-

In [37]:
instructor = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/INSTRUCTOR_DATA_TABLE.csv")\
    .withColumn("CREATED_DATE", to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE", to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

instructor.printSchema()
instructor.createOrReplaceTempView("instructor")
instructor.show(3)

root
 |-- INSTRUCTOR_ID: integer (nullable = true)
 |-- SALUTATION: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- STREET_ADDRESS: string (nullable = true)
 |-- ZIP: integer (nullable = true)
 |-- PHONE: integer (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+-------------+----------+----------+---------+--------------+-----+----------+----------+------------+-----------+-------------+
|INSTRUCTOR_ID|SALUTATION|FIRST_NAME|LAST_NAME|STREET_ADDRESS|  ZIP|     PHONE|CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+-------------+----------+----------+---------+--------------+-----+----------+----------+------------+-----------+-------------+
|          101|        Mr|   Fernand|    Hanks| 100 East 87th|10015|2125551212|  ESILVEST|  2007-01-02|   ESILVEST|   2007-01-02|
|          10

In [38]:
student = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/STUDENT_DATA_TABLE.csv")\
    .withColumn("REGISTRATION_DATE", to_date(col('REGISTRATION_DATE'), "dd-MMM-yy"))\
    .withColumn("CREATED_DATE",      to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE",     to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

student.cache()
student.printSchema()
student.createOrReplaceTempView("student")
student.show(3)

root
 |-- STUDENT_ID: integer (nullable = true)
 |-- SALUTATION: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- STREET_ADDRESS: string (nullable = true)
 |-- ZIP: integer (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- EMPLOYER: string (nullable = true)
 |-- REGISTRATION_DATE: date (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+----------+----------+----------+---------+------------------+-----+------------+---------------+-----------------+-----------+------------+-----------+-------------+
|STUDENT_ID|SALUTATION|FIRST_NAME|LAST_NAME|    STREET_ADDRESS|  ZIP|       PHONE|       EMPLOYER|REGISTRATION_DATE| CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+----------+----------+----------+---------+------------------+-----+------------+---------------+----------------

In [39]:
enrollment = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/ENROLLMENT_DATA_TABLE.csv")\
    .withColumn("CREATED_DATE", to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE", to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

enrollment.cache()
enrollment.printSchema()
enrollment.createOrReplaceTempView("enrollment")
enrollment.show(3)

root
 |-- STUDENT_ID: integer (nullable = true)
 |-- SECTION_ID: integer (nullable = true)
 |-- ENROLL_DATE: string (nullable = true)
 |-- FINAL_GRADE: integer (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+----------+----------+-----------+-----------+----------+------------+-----------+-------------+
|STUDENT_ID|SECTION_ID|ENROLL_DATE|FINAL_GRADE|CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+----------+----------+-----------+-----------+----------+------------+-----------+-------------+
|       215|       146|  13-FEB-07|       null|  DSCHERER|  2007-12-14|   BROSENZW|   2007-01-05|
|       215|       156|  13-FEB-07|       null|  DSCHERER|  2007-12-14|   BROSENZW|   2007-01-05|
|       216|       154|  13-FEB-07|       null|  DSCHERER|  2007-12-14|   BROSENZW|   2007-01-05|
+----------+----------+-----------+-----------+----------+-----

In [40]:
grade = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/GRADE_DATA_TABLE.csv")\
    .withColumn("CREATED_DATE", to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE", to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

grade.printSchema()
grade.createOrReplaceTempView("grade")
grade.show(3)

root
 |-- STUDENT_ID: integer (nullable = true)
 |-- SECTION_ID: integer (nullable = true)
 |-- GRADE_TYPE_CODE: string (nullable = true)
 |-- GRADE_CODE_OCCURRENCE: integer (nullable = true)
 |-- NUMERIC_GRADE: integer (nullable = true)
 |-- COMMENTS: string (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+----------+----------+---------------+---------------------+-------------+--------+----------+------------+-----------+-------------+
|STUDENT_ID|SECTION_ID|GRADE_TYPE_CODE|GRADE_CODE_OCCURRENCE|NUMERIC_GRADE|COMMENTS|CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+----------+----------+---------------+---------------------+-------------+--------+----------+------------+-----------+-------------+
|       111|       133|             PA|                    6|           80|    null|  CBRENNAN|  2007-02-11|     JAYCAF|   2007-02-11|
|       111

Need to make sure 31-DEC-98 is converted to 1998-12-31, not 2098-12-31.

* [spark to_date function - how to convert 31-DEC-98 to 1998-12-31 not 2098-12-31](https://stackoverflow.com/questions/71182230)

In [41]:
grade_type = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/GRADE_TYPE_DATA_TABLE.csv")\
    .withColumn("CREATED_DATE", to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE", to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

grade_type.printSchema()
grade_type.createOrReplaceTempView("grade_type")
grade_type.show(3)

root
 |-- GRADE_TYPE_CODE: string (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+---------------+-----------+----------+------------+-----------+-------------+
|GRADE_TYPE_CODE|DESCRIPTION|CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+---------------+-----------+----------+------------+-----------+-------------+
|             FI|      Final|  MCAFFREY|  1998-12-31|   MCAFFREY|   1998-12-31|
|             HM|   Homework|  MCAFFREY|  1998-12-31|   MCAFFREY|   1998-12-31|
|             MT|    Midterm|  MCAFFREY|  1998-12-31|   MCAFFREY|   1998-12-31|
+---------------+-----------+----------+------------+-----------+-------------+
only showing top 3 rows



In [42]:
zipcode = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/ZIPCODE_DATA_TABLE.csv")\
    .withColumn("CREATED_DATE", to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE", to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

zipcode.printSchema()
zipcode.createOrReplaceTempView("zipcode")
zipcode.show(3)

root
 |-- ZIP: integer (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+-----+----------------+-----+----------+------------+-----------+-------------+
|  ZIP|            CITY|STATE|CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+-----+----------------+-----+----------+------------+-----------+-------------+
|11101|Long Island City|   NY|  AMORRISO|  2007-08-03|   AMORRISO|   2007-11-24|
|11102|         Astoria|   NY|  AMORRISO|  2007-08-03|   AMORRISO|   2007-11-24|
|11103|         Astoria|   NY|  AMORRISO|  2007-08-03|   AMORRISO|   2007-11-24|
+-----+----------------+-----+----------+------------+-----------+-------------+
only showing top 3 rows



In [43]:
employee = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/EMPLOYEE_DATA_TABLE.csv")

employee.printSchema()
employee.createOrReplaceTempView("employee")
employee.show(3)

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- NAME: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- TITLE: string (nullable = true)

+-----------+------+------+---------+
|EMPLOYEE_ID|  NAME|SALARY|    TITLE|
+-----------+------+------+---------+
|          1|  John|  1000|  Analyst|
|          2|  Mary|  2000|  Manager|
|          3|Stella|  5000|President|
+-----------+------+------+---------+
only showing top 3 rows



---
# Analytics Functions

The general syntax of analytical functions is **```analytic_function([arguments]) OVER (analytic_clause)```**

```OVER``` keyword indicates that the function operates after the results of the FROM, WHERE, GROUP BY, and HAVING clauses have been formed.

**analytic_clause** can contain three other clauses: ```QUERY_PARTITIONING```, ```ORDER_BY```, or ```WINDOWING```.<br>
```[query_partition_clause] [order_by_clause [windowing_clause]]```

<img src="./image/analytics_function_query_execution_steps.jpg" align="left" width=400/>

---
# Analytcis function vs GROUP BY

* [How X in SELECT X, AGG(Z) OVER (PARTITION X) is handled in analytics SQL](https://stackoverflow.com/questions/71201623/sql-how-x-in-select-x-aggz-over-partition-x-is-handled-in-analytics-sql)

The SQL engine finds X column in SELECT statement, and check if it exists in the PARITION BY clause. If exists, the value of the X is applied in the PARTITION BY part. This is similar to the correlated subquery and GROUP BY SQL ```SELECT X, AGG(Z) FROM T GROUP BY X```.

In [159]:
query = """
SELECT DISTINCT
    grade_type_code,
    MAX(numeric_grade) OVER (PARTITION BY grade_type_code) as maximum,
    MIN(numeric_grade) OVER (PARTITION BY grade_type_code) as minimum,
    ROUND(AVG(numeric_grade) OVER (PARTITION BY grade_type_code),2) as mean,
    ROUND(VARIANCE(numeric_grade) OVER (PARTITION BY grade_type_code),2) as var,
    ROUND(STDDEV(numeric_grade) OVER (PARTITION BY grade_type_code),2) as std,
    COUNT(numeric_grade) OVER (PARTITION BY grade_type_code) as cnt
FROM
    grade
ORDER BY
    grade_type_code
"""
spark.sql(query).show(truncate=False)

+---------------+-------+-------+-----+-----+----+---+
|grade_type_code|maximum|minimum|mean |var  |std |cnt|
+---------------+-------+-------+-----+-----+----+---+
|FI             |99     |76     |85.77|46.99|6.86|205|
|HM             |99     |70     |86.12|67.27|8.2 |798|
|MT             |99     |76     |87.08|43.15|6.57|204|
|PA             |99     |70     |86.32|56.43|7.51|394|
|PJ             |99     |76     |87.0 |48.2 |6.94|21 |
|QZ             |99     |73     |86.75|59.97|7.74|382|
+---------------+-------+-------+-----+-----+----+---+



In [160]:
query = """
SELECT 
    grade_type_code,
    MAX(numeric_grade) as maximum,
    MIN(numeric_grade) as minimum,
    ROUND(AVG(numeric_grade),2) as mean,
    ROUND(VARIANCE(numeric_grade), 2) as var,
    ROUND(STDDEV(numeric_grade), 2) as std,
    COUNT(*) as cnt
FROM
    grade
GROUP BY
    grade_type_code
ORDER BY 
    grade_type_code
"""
spark.sql(query).show(truncate=False)

+---------------+-------+-------+-----+-----+----+---+
|grade_type_code|maximum|minimum|mean |var  |std |cnt|
+---------------+-------+-------+-----+-----+----+---+
|FI             |99     |76     |85.77|46.99|6.86|205|
|HM             |99     |70     |86.12|67.27|8.2 |798|
|MT             |99     |76     |87.08|43.15|6.57|204|
|PA             |99     |70     |86.32|56.43|7.51|394|
|PJ             |99     |76     |87.0 |48.2 |6.94|21 |
|QZ             |99     |73     |86.75|59.97|7.74|382|
+---------------+-------+-------+-----+-----+----+---+



---
# Pivot

Transform from Long/Stack Format to Wide/Unstack Format. 

1. Find the ID (keys) that identifyes an class instance that will be a row in Wide Format.
2. Create (ID, attribute, value) Long Format.
3. ```PIVOT (FOR attribute IN (<attributes>))```


* [PIVOT Clause](https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-pivot.html)

## Syntax
```
PIVOT (
    aggregate_expression(value) [ AS aggregate_expression_alias ]
    [ , ... ]   /* Continue with , for multipel aggregations */
    FOR attributes IN ( 
        attribute_list 
    )
) 
```


Find the number of classes starting at each location.

1. Find the ```location``` as **ID**.
2. Create (ID, attribute, value) Long Format table.
3. PIVOT (Long to Wide Transformation) that generates rows where each row is an class instance with attributes.


<img src="./image/pivot.jpg" align="left" width=600/>

In [73]:
query = """
SELECT
    location,
    date_format(start_date_time, "EEE") AS day_text,
    dayofweek(start_date_time) AS day_num
FROM
    section
ORDER BY location
"""
spark.sql(query).show()

+--------+--------+-------+
|location|day_text|day_num|
+--------+--------+-------+
|    H310|     Tue|      3|
|    L206|     Tue|      3|
|    L210|     Tue|      3|
|    L210|     Mon|      2|
|    L210|     Tue|      3|
|    L210|     Fri|      6|
|    L210|     Sat|      7|
|    L210|     Tue|      3|
|    L210|     Sat|      7|
|    L210|     Wed|      4|
|    L210|     Mon|      2|
|    L210|     Sun|      1|
|    L211|     Thu|      5|
|    L211|     Tue|      3|
|    L211|     Sat|      7|
|    L214|     Sat|      7|
|    L214|     Fri|      6|
|    L214|     Thu|      5|
|    L214|     Sun|      1|
|    L214|     Thu|      5|
+--------+--------+-------+
only showing top 20 rows



In [66]:
query = """
WITH location_day_cnt AS (
    SELECT
        location,
        upper(date_format(start_date_time, "EEE")) AS day,
        COUNT(*) AS cnt
    FROM
        section
    GROUP BY
        location, upper(date_format(start_date_time, "EEE"))
    ORDER BY location
)
SELECT
    *
FROM
    location_day_cnt
PIVOT (
    SUM(cnt)
    FOR day IN (
        'MON' AS MON,
        'TUE' AS TUE,
        'WED' AS WED,
        'THU' AS THU,
        'FRI' AS FRI,
        'SAT' AS SAT,
        'SUN' AS SUN
    )
)
ORDER BY
    location
"""
spark.sql(query).show()

+--------+----+----+----+----+----+----+----+
|location| MON| TUE| WED| THU| FRI| SAT| SUN|
+--------+----+----+----+----+----+----+----+
|    H310|null|   1|null|null|null|null|null|
|    L206|null|   1|null|null|null|null|null|
|    L210|   2|   3|   1|null|   1|   2|   1|
|    L211|null|   1|null|   1|null|   1|null|
|    L214|   2|   2|null|   2|   1|   4|   4|
|    L500|   1|   1|null|null|null|null|null|
|    L507|   4|   3|   3|null|   1|   3|   1|
|    L509|   4|   5|   3|   2|   1|   4|   6|
|    L511|null|null|null|null|null|   1|null|
|    M200|   1|null|null|null|null|null|null|
|    M311|   1|null|null|null|null|   1|   1|
|    M500|null|null|null|null|null|   1|null|
+--------+----+----+----+----+----+----+----+



---
# MAX/FIRST_VALUE

Student with best grade in (course, section)

### GROUP BY approach

1. Create a joined table of (grade, section).
2. GROUP BY (course_no, section_id) and get MAX(numeric_grade).
3. Annotate with course.description, student.first_name, student.last_name.

In [79]:
query = """
WITH course_section_max_grade AS (
    SELECT
        s.course_no,
        g.section_id,
        MAX(g.numeric_grade) as best_grade
    FROM
        grade g
        INNER JOIN section s ON s.section_id = g.section_id
    WHERE
        g.grade_type_code = 'FI'
    GROUP BY
        s.course_no,
        g.section_id
    ORDER BY
        s.course_no,
        g.section_id
)
SELECT
    m.course_no,
    (SELECT MAX(c.description) FROM course c WHERE c.course_no = m.course_no) AS course_name,
    m.section_id,
    m.best_grade,
    concat(t.first_name, ' ', t.last_name) AS student_name
FROM
    course_section_max_grade m
    INNER JOIN grade g 
        ON g.section_id = m.section_id 
        AND g.numeric_grade = m.best_grade
    INNER JOIN student t
        ON t.student_id = g.student_id
WHERE
    g.grade_type_code = 'FI'
ORDER BY 
    m.course_no,
    m.section_id
"""
spark.sql(query).show()

+---------+--------------------+----------+----------+-----------------+
|course_no|         course_name|section_id|best_grade|     student_name|
+---------+--------------------+----------+----------+-----------------+
|       10| Technology Concepts|        80|        91|      Jeff Runyan|
|       20|Intro to Informat...|        81|        92|    Laetia Enison|
|       20|Intro to Informat...|        83|        99|Daniel Wicelinski|
|       20|Intro to Informat...|        84|        99|        J. Segall|
|       25|Intro to Programming|        85|        92|     Gerard Biers|
|       25|Intro to Programming|        86|        99|   Jeffrey Citron|
|       25|Intro to Programming|        87|        99|  Pierre Radicola|
|       25|Intro to Programming|        89|        99|        M. Diokno|
|       25|Intro to Programming|        90|        92|    Eric Da Silva|
|       25|Intro to Programming|        91|        77|     Jose Benitez|
|       25|Intro to Programming|        92|        

### Analytics approach

1. Create a joined table of (course, section, grade).
2. Order DESC by grade in each partition (course_no, section_id) and pick the FIRST_VLAUE.

In [86]:
query = """
WITH course_section_grade AS (
    SELECT
        c.course_no AS course_no,
        c.description AS course_name,
        g.section_id AS section_id,
        g.numeric_grade as grade,
        t.student_id,
        (t.first_name || ' ' || t.last_name) as student_name
    FROM
        grade g
        INNER JOIN section s
            ON s.section_id = g.section_id
        INNER JOIN course c
            ON s.course_no = c.course_no
        INNER JOIN student t
            ON t.student_id = g.student_id
    WHERE
        g.grade_type_code = 'FI'
    ORDER BY
        c.course_no,
        g.section_id,
        g.numeric_grade
)
SELECT     
    course_no,
    course_name,
    section_id,
    FIRST_VALUE(grade) OVER (PARTITION BY course_no, section_id ORDER BY grade DESC) as best_grade,
    FIRST_VALUE(student_name) OVER (PARTITION BY course_no, section_id ORDER BY grade DESC) as best_student
FROM
    course_section_grade csg
ORDER BY
    course_no,
    section_id
"""
spark.sql(query).show(truncate=False)

+---------+----------------------------+----------+----------+-----------------+
|course_no|course_name                 |section_id|best_grade|best_student     |
+---------+----------------------------+----------+----------+-----------------+
|10       |Technology Concepts         |80        |91        |Jeff Runyan      |
|20       |Intro to Information Systems|81        |92        |Laetia Enison    |
|20       |Intro to Information Systems|81        |92        |Laetia Enison    |
|20       |Intro to Information Systems|81        |92        |Laetia Enison    |
|20       |Intro to Information Systems|83        |99        |Daniel Wicelinski|
|20       |Intro to Information Systems|83        |99        |Daniel Wicelinski|
|20       |Intro to Information Systems|84        |99        |J. Segall        |
|20       |Intro to Information Systems|84        |99        |J. Segall        |
|25       |Intro to Programming        |85        |92        |Gerard Biers     |
|25       |Intro to Programm

---
# Rank/DENSE_RANK

The RANK() function assigns a rank to each row based on a provided column. 

```RANK() OVER (PARTITION BY columns ORDER BY columns ASC|DESC)```

Rank of students based on the final grade

In [107]:
query = """
WITH course_section_grade AS (
    SELECT
        c.course_no AS course_no,
        c.description AS course_name,
        g.section_id AS section_id,
        g.numeric_grade as grade,
        t.student_id,
        (t.first_name || ' ' || t.last_name) as student_name
    FROM
        grade g
        INNER JOIN section s
            ON s.section_id = g.section_id
        INNER JOIN course c
            ON s.course_no = c.course_no
        INNER JOIN student t
            ON t.student_id = g.student_id
    WHERE
        g.grade_type_code = 'FI'
    ORDER BY
        c.course_no,
        g.section_id,
        g.numeric_grade
)
SELECT
    course_no,
    course_name,
    section_id,
    student_name,
    grade,
    NVL(LAG(grade) OVER (PARTITION BY course_no ORDER BY grade DESC) - grade, 'NA') AS delta, 
    DENSE_RANK() OVER (PARTITION BY course_no ORDER BY grade DESC) AS ranking,
    ROW_NUMBER() OVER (PARTITION BY course_no ORDER BY grade DESC) AS rownum
FROM
    course_section_grade
ORDER BY
    course_no,
    ranking,
    rownum
"""
spark.sql(query).show(truncate=False)

+---------+----------------------------+----------+-------------------+-----+-----+-------+------+
|course_no|course_name                 |section_id|student_name       |grade|delta|ranking|rownum|
+---------+----------------------------+----------+-------------------+-----+-----+-------+------+
|10       |Technology Concepts         |80        |Jeff Runyan        |91   |NA   |1      |1     |
|20       |Intro to Information Systems|83        |Daniel Wicelinski  |99   |NA   |1      |1     |
|20       |Intro to Information Systems|84        |J. Segall          |99   |0    |1      |2     |
|20       |Intro to Information Systems|81        |Laetia Enison      |92   |7    |2      |3     |
|20       |Intro to Information Systems|81        |J. Landry          |91   |1    |3      |4     |
|20       |Intro to Information Systems|83        |Michael Carcia     |90   |1    |4      |5     |
|20       |Intro to Information Systems|84        |Roy Limate         |88   |2    |5      |6     |
|20       

---
# NTILE

```NTILE(number_of_bins) OVER (PARTITION BY columns ORDER BY columns [DESC|ASC]```

Top 5% (20 TILES) highest grade student in the entire students.

In [122]:
query = """
WITH course_section_grade AS (
    SELECT
        c.course_no AS course_no,
        c.description AS course_name,
        g.section_id AS section_id,
        g.numeric_grade as grade,
        t.student_id,
        (t.first_name || ' ' || t.last_name) as student_name
    FROM
        grade g
        INNER JOIN section s
            ON s.section_id = g.section_id
        INNER JOIN course c
            ON s.course_no = c.course_no
        INNER JOIN student t
            ON t.student_id = g.student_id
    WHERE
        g.grade_type_code = 'FI'
    ORDER BY
        c.course_no,
        g.section_id,
        g.numeric_grade
)
SELECT 
    *
FROM (
    SELECT
        student_name,
        grade,
        NTILE(20) OVER (ORDER BY grade DESC) AS quantile
    FROM
        course_section_grade
) q
WHERE 
    q.quantile = 1 OR q.quantile = 20
"""
spark.sql(query).show(truncate=False)

2022-02-21 14:55:57,631 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------------+-----+--------+
|student_name          |grade|quantile|
+----------------------+-----+--------+
|Daniel Wicelinski     |99   |1       |
|J. Segall             |99   |1       |
|Regina Bose           |99   |1       |
|Yvonne Williams       |99   |1       |
|Salewa Zuckerberg     |99   |1       |
|Daniel Wicelinski     |99   |1       |
|Judy Cahouet          |99   |1       |
|Catherine Frangopoulos|99   |1       |
|Jeffrey Citron        |99   |1       |
|Pierre Radicola       |99   |1       |
|M. Diokno             |99   |1       |
|Arlyne Sheppard       |76   |20      |
|Janet Jung            |76   |20      |
|Daniel Ordes          |76   |20      |
|Charles Caro          |76   |20      |
|Anil Kulina           |76   |20      |
|James Reed            |76   |20      |
|Angel Cook            |76   |20      |
|Ricardo Kurtz         |76   |20      |
|Catherine Frangopoulos|76   |20      |
+----------------------+-----+--------+
only showing top 20 rows



# MODE

The value that occurs with the greatest frequency. ```STATS_MODE(expr)``` does not exist in SparkSQL.

Find the most frequent numeric_grade for each grade_type_code in the grade table.

In [131]:
query = """
SELECT
    grade_type_code,
    numeric_grade AS numeric_grade,
    COUNT(numeric_grade) AS occurrences
FROM
    grade
GROUP BY 
    grade_type_code, numeric_grade
ORDER BY
    grade_type_code,
    numeric_grade
"""
spark.sql(query).show(truncate=False)

+---------------+-------------+-----------+
|grade_type_code|numeric_grade|occurrences|
+---------------+-------------+-----------+
|FI             |76           |28         |
|FI             |77           |23         |
|FI             |83           |18         |
|FI             |84           |18         |
|FI             |85           |25         |
|FI             |88           |22         |
|FI             |90           |16         |
|FI             |91           |16         |
|FI             |92           |20         |
|FI             |99           |19         |
|HM             |70           |2          |
|HM             |71           |2          |
|HM             |72           |4          |
|HM             |73           |23         |
|HM             |74           |44         |
|HM             |75           |39         |
|HM             |76           |36         |
|HM             |77           |21         |
|HM             |78           |6          |
|HM             |79           |2

In [132]:
query = """
WITH mode_per_grade_type AS (
    SELECT
        grade_type_code,
        numeric_grade AS numeric_grade,
        COUNT(numeric_grade) AS occurrences
    FROM
        grade
    GROUP BY 
        grade_type_code, numeric_grade
    ORDER BY
        grade_type_code,
        numeric_grade
)
SELECT
    grade_type_code,
    MAX(occurrences) mode
FROM
    mode_per_grade_type
GROUP BY 
    grade_type_code
ORDER BY
    grade_type_code,
    mode DESC
"""
spark.sql(query).show(truncate=False)

+---------------+----+
|grade_type_code|mode|
+---------------+----+
|FI             |28  |
|HM             |59  |
|MT             |28  |
|PA             |31  |
|PJ             |4   |
|QZ             |37  |
+---------------+----+



---
# Windowing
```
SELECT
    <Analytics Function> OVER (
        PARTITION BY <partition_column>
        ORDER BY <order_column>
        ROWS BETWEEN <start> AND <end>
    ) AS <name>
FROM
    <TABLE> 
```

ORDER BY in a windowing clause is a mandatory clause that determines the order in which the rows are sorted. Based on this order, the starting and ending points of the window are defined.

### Start/End of the window

* UNBOUNDED PRECEDING :<br>
The window starts at the first row of the partition, or the whole result set if no partitioning clause is used. Only available for start points.
* UNBOUNDED FOLLOWING :<br>
The window ends at the last row of the partition, or the whole result set if no partitioning clause is used. Only available for end points.
* CURRENT ROW :<br>
The window starts or ends at the current row. Can be used as start or end point.
* value_expr PRECEDING :  (e.g. 1 PRECEEDING)<br>
A physical or logical offset before the current row using a constant or expression that evaluates to a positive numerical value. When used with RANGE, it can also be an interval literal if the order_by_clause uses a DATE column.
* value_expr FOLLOWING :  (e.g. 1 FOLLOWING)<br>
As above, but an offset after the current row.



From the top to the current, ```ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW``` 

<img src="./image/window.jpg" align="left" />

## Cumulative sum

In [168]:
query = """
SELECT
    course_no,
    revenue,
    SUM(revenue) OVER (ORDER BY course_no, revenue ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumsum
FROM
    course_revenue
ORDER BY 
    course_no,
    revenue
"""
spark.sql(query).show(truncate=False)

+---------+-------+------+
|course_no|revenue|cumsum|
+---------+-------+------+
|10       |1195   |1195  |
|20       |10755  |11950 |
|25       |53775  |65725 |
|100      |15535  |81260 |
|120      |27485  |108745|
|122      |28680  |137425|
|124      |9560   |146985|
|125      |9560   |156545|
|130      |9560   |166105|
|132      |2390   |168495|
|134      |2390   |170885|
|135      |4380   |175265|
|140      |17925  |193190|
|142      |8365   |201555|
|145      |2390   |203945|
|146      |3585   |207530|
|147      |5975   |213505|
|204      |1195   |214700|
|230      |15330  |230030|
|240      |14235  |244265|
+---------+-------+------+
only showing top 20 rows



2022-02-21 17:24:05,785 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


---
# Stop Spark Session

In [169]:
spark.stop()



# Cleanup

In [170]:
del spark
gc.collect()

9750